# Steps
- Extract drug names for prostate cancers - (TCGA_DESC = PRAD)
- Filter extracted drugs based on:
    - MAX_CONC <= 10uM - Why?
- Extract unique list of drugs, if not already done
- Find canonical SMILES of each drug from PubChem
- Standardize canonical SMILES using rdkit
- Export the processed file to reference_drug.ipynb notebook, follow the fingerprint construction pipeline.

In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('GDSC2_fitted_dose_response_27Oct23.csv')
print(df.shape)
df.head()

(242036, 19)


,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,343,15946310,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.463887,0.930220,0.089052,0.433123
1,GDSC2,343,15946548,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869455,0.614970,0.111351,-1.421100
2,GDSC2,343,15946830,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360586,0.791072,0.142855,-0.599569
3,GDSC2,343,15947087,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.044940,0.592660,0.135539,-1.516647
4,GDSC2,343,15947369,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741991,0.734047,0.128059,-0.807232


In [53]:
df_prad = df.loc[df['TCGA_DESC']=='PRAD']
print(df_prad.shape)
df_prad.head()

(1676, 19)


,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
167,GDSC2,343,15987973,905934,PC-3,SIDM00088,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.796951,0.861456,0.085586,0.251780
168,GDSC2,343,15988254,905935,DU-145,SIDM00120,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-2.918755,0.803630,0.103836,-0.359006
361,GDSC2,343,16039919,907788,LNCaP-Clone-FGC,SIDM00683,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.589240,0.745884,0.090794,-0.724064
608,GDSC2,343,16102293,924100,22RV1,SIDM00499,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-2.903667,0.783025,0.106837,-0.350791
863,GDSC2,343,16164935,1299075,VCaP,SIDM01077,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.911319,0.866654,0.061343,0.189511


In [54]:
df_prad_filtered = df_prad.loc[df_prad['MAX_CONC']<=10]
print(df_prad_filtered.shape)
df_prad_filtered.head()

(1517, 19)


,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
167,GDSC2,343,15987973,905934,PC-3,SIDM00088,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.796951,0.861456,0.085586,0.251780
168,GDSC2,343,15988254,905935,DU-145,SIDM00120,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-2.918755,0.803630,0.103836,-0.359006
361,GDSC2,343,16039919,907788,LNCaP-Clone-FGC,SIDM00683,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.589240,0.745884,0.090794,-0.724064
608,GDSC2,343,16102293,924100,22RV1,SIDM00499,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-2.903667,0.783025,0.106837,-0.350791
863,GDSC2,343,16164935,1299075,VCaP,SIDM01077,PRAD,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.911319,0.866654,0.061343,0.189511


In [55]:
print(len(set(df_prad_filtered['DRUG_NAME'])))
print(set(df_prad_filtered['DRUG_NAME']))

247
{'Bosutinib', 'AZ6102', 'Entinostat', 'BIBR-1532', 'MK-8776', 'Zoledronate', 'GSK343', 'AZD3759', 'WEHI-539', 'Oxaliplatin', 'MIRA-1', 'Avagacestat', '743380', 'BDILV000379a', 'Ribociclib', 'Rucaparib', 'BMS-754807', 'Taselisib', 'AZD6738', 'Dactolisib', 'BMS-345541', 'IRAK4_4710', 'OTX015', 'PLX-4720', 'Savolitinib', 'IAP_5620', 'JAK_8517', 'Sepantronium bromide', 'AZD2014', 'GSK1904529A', 'ABT737', 'Luminespib', 'LMB_AB3', 'SGC-CBP30', 'PCI-34051', 'BDOCA000347a', 'Erlotinib', 'AZ960', 'WZ4003', '50869', 'AZD8186', 'Mycophenolic acid', 'Navitoclax', 'Bortezomib', 'RO-3306', 'CT7033-2', 'AZD1208', 'TAF1_5496', 'Topotecan', 'Alpelisib', 'LMB_AB1', '123829', 'LMP744', 'AZD5363', 'JNK Inhibitor VIII', 'JQ1', 'AGI-6780', 'Motesanib', 'Sapitinib', 'AT13148', 'Ruxolitinib', 'OF-1', 'SB590885', 'P22077', 'VE821', 'Obatoclax Mesylate', 'Afatinib', 'GSK3337463A', 'Schweinfurthin A', 'GSK2830371', 'Staurosporine', '765771', 'XAV939', 'Pictilisib', 'SN-38', 'I-BET-762', 'N27922-53-1', 'Entos

In [56]:
df_prad_filtered_drugs = pd.DataFrame({'DRUG_NAME': sorted(list(set(df_prad_filtered['DRUG_NAME'])))})
df_prad_filtered_drugs

,DRUG_NAME
0,123138
1,123829
2,150412
3,50869
4,615590
...,...
242,Wee1 Inhibitor
243,Wnt-C59
244,XAV939
245,ZM447439


In [57]:
# df_prad_filtered_drugs.to_csv('gdsc_drugs.csv', index=False)

### There are two types of identifiers present for the drugs - name, cid
- Separate the two types - and extract the SMILES separately for the two groups

In [58]:
df_drugs = pd.read_csv('gdsc_drugs.csv')
df_drugs

,DRUG_NAME
0,123138
1,123829
2,150412
3,50869
4,615590
...,...
242,Wee1 Inhibitor
243,Wnt-C59
244,XAV939
245,ZM447439


In [59]:
import re
from pubchem import name2smiles, cid2smiles
canonical_SMILES = [None] * len(df_drugs)
for i in range(len(df_drugs['DRUG_NAME'])):
    if re.match('^\d', df_drugs['DRUG_NAME'][i]):
        canonical_SMILES[i] = cid2smiles(df_drugs['DRUG_NAME'][i])
    else:
        canonical_SMILES[i] = name2smiles(df_drugs['DRUG_NAME'][i])
    

Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 503
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status code 404
Request failed with status c

In [61]:
df_drugs['canonical_SMILES'] = canonical_SMILES
df_drugs_drop_na = df_drugs.dropna().reset_index(drop=True)
df_drugs_drop_na

,DRUG_NAME,canonical_SMILES
0,123138,C[CH2]
1,123829,CC(C[Se]CC(=O)NCCS(=O)(=O)O)C1CCC2C1(C(CC3C2C(...
2,150412,CC(CO)(CO)NCC1=CC2=C(C=C1)C3=C(C=C2)C4=CC=CC=C...
3,50869,C1=CC=C(C=C1)OCCCCCCNN
4,615590,CNC(=O)CSC1=C(NC2=CC=CC=C21)C3=CC=CS3
...,...,...
202,Wee1 Inhibitor,C1=CC=C(C(=C1)C2=CC3=C(C4=C(N3)C=CC(=C4)O)C5=C...
203,Wnt-C59,CC1=NC=CC(=C1)C2=CC=C(C=C2)CC(=O)NC3=CC=C(C=C3...
204,XAV939,C1CSCC2=C1N=C(NC2=O)C3=CC=C(C=C3)C(F)(F)F
205,ZM447439,COC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=C(C=C3)NC(=O)...


In [62]:
# Standardize the smiles using rdkit
from rdkit_transformation import standardize
standardized_SMILES = []
for smiles in df_drugs_drop_na['canonical_SMILES']:
    standardized_SMILES.append(standardize(smiles))

In [63]:
df_drugs_drop_na['standardized_SMILES'] = standardized_SMILES
df_drugs_drop_na

,DRUG_NAME,canonical_SMILES,standardized_SMILES
0,123138,C[CH2],[CH2]C
1,123829,CC(C[Se]CC(=O)NCCS(=O)(=O)O)C1CCC2C1(C(CC3C2C(...,CC(C[Se]CC(=O)NCCS(=O)(=O)O)C1CCC2C3C(O)CC4CC(...
2,150412,CC(CO)(CO)NCC1=CC2=C(C=C1)C3=C(C=C2)C4=CC=CC=C...,CC(CO)(CO)NCc1ccc2c(ccc3c4ccccc4ccc23)c1
3,50869,C1=CC=C(C=C1)OCCCCCCNN,NNCCCCCCOc1ccccc1
4,615590,CNC(=O)CSC1=C(NC2=CC=CC=C21)C3=CC=CS3,CNC(=O)CSc1c(-c2cccs2)[nH]c2ccccc12
...,...,...,...
202,Wee1 Inhibitor,C1=CC=C(C(=C1)C2=CC3=C(C4=C(N3)C=CC(=C4)O)C5=C...,O=C1NC(=O)c2c1c(-c1ccccc1Cl)cc1[nH]c3ccc(O)cc3c21
203,Wnt-C59,CC1=NC=CC(=C1)C2=CC=C(C=C2)CC(=O)NC3=CC=C(C=C3...,Cc1cc(-c2ccc(CC(=O)Nc3ccc(-c4cccnc4)cc3)cc2)ccn1
204,XAV939,C1CSCC2=C1N=C(NC2=O)C3=CC=C(C=C3)C(F)(F)F,O=c1[nH]c(-c2ccc(C(F)(F)F)cc2)nc2c1CSCC2
205,ZM447439,COC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=C(C=C3)NC(=O)...,COc1cc2c(Nc3ccc(NC(=O)c4ccccc4)cc3)ncnc2cc1OCC...


In [65]:
# df_drugs_drop_na.to_csv('gdsc_drugs+SMILES.csv', index=False)